In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Asthma"
cohort = "GSE205151"

# Input paths
in_trait_dir = "../../input/GEO/Asthma"
in_cohort_dir = "../../input/GEO/Asthma/GSE205151"

# Output paths
out_data_file = "../../output/preprocess/Asthma/GSE205151.csv"
out_gene_data_file = "../../output/preprocess/Asthma/gene_data/GSE205151.csv"
out_clinical_data_file = "../../output/preprocess/Asthma/clinical_data/GSE205151.csv"
json_path = "../../output/preprocess/Asthma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Functional Immunophenotyping of Children with Critical Status Asthmaticus Identifies Differential Gene Expression Responses in Neutrophils Exposed to a Poly(I:C) Stimulus"
!Series_summary	"We determined whether we could identify clusters of children with critical asthma by functional immunophenotyping using an intracellular viral analog stimulus."
!Series_summary	"We performed a single-center, prospective, observational cohort study of 43 children ages 6 – 17 years admitted to a pediatric intensive care unit for an asthma attack between July 2019 to February 2021."
!Series_overall_design	"Neutrophils were isolated from children, stimulated overnight with LyoVec poly(I:C), and mRNA was analyzed using a targeted Nanostring immunology array. Network analysis of the differentially expressed transcripts for the paired LyoVec poly(I:C) samples was performed."
Sample Characteristics Dictionary:
{0: ['polyic_stimulation: Unstimulated', 'polyic_stimulation

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import gzip
import io

# Analyze the available information
# From the background information, we can determine this is a gene expression dataset (Nanostring immunology array)
is_gene_available = True

# Look at what files are available in the input directory
available_files = os.listdir(in_cohort_dir)
print(f"Available files in {in_cohort_dir}: {available_files}")

# For trait - using 'cluster' as a potential proxy for asthma severity/subtypes
trait_row = 1  # The row with 'cluster' information
def convert_trait(value):
    if pd.isna(value):
        return None
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    # Convert to binary (0 for cluster 1, 1 for cluster 2)
    if value == '1':
        return 0
    elif value == '2':
        return 1
    else:
        return None

# Age data is not available in the provided information
age_row = None
def convert_age(value):
    return None

# Gender data is not available in the provided information
gender_row = None
def convert_gender(value):
    return None

# Initial validation to see if we should continue processing this dataset
is_trait_available = (trait_row is not None)
validated = validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# If clinical data is available, extract and save it
if trait_row is not None:
    try:
        # Find and extract the series matrix file which should contain clinical information
        series_matrix_file = None
        for file in available_files:
            if "series_matrix" in file.lower():
                series_matrix_file = os.path.join(in_cohort_dir, file)
                break
        
        if series_matrix_file:
            print(f"Found series matrix file: {series_matrix_file}")
            
            # Read and parse the gzipped series matrix file
            clinical_data = None
            sample_ids = []
            sample_characteristics = {}
            
            with gzip.open(series_matrix_file, 'rt') as f:
                lines = f.readlines()
                
                # Extract sample IDs
                for line in lines:
                    if line.startswith('!Sample_geo_accession'):
                        sample_ids = line.strip().split('\t')[1:]
                        break
                
                # Extract sample characteristics
                row_idx = 0
                for line in lines:
                    if line.startswith('!Sample_characteristics_ch1'):
                        char_values = line.strip().split('\t')[1:]
                        sample_characteristics[row_idx] = char_values
                        row_idx += 1
            
            # Create a DataFrame from the extracted sample characteristics
            clinical_data = pd.DataFrame(sample_characteristics, index=sample_ids).T
            
            # Display what we've extracted
            print("Extracted sample characteristics:")
            for row_idx, values in sample_characteristics.items():
                unique_values = list(set([v for v in values if pd.notna(v)]))
                print(f"Row {row_idx}: {unique_values[:5]}{'...' if len(unique_values) > 5 else ''}")
            
            # Select and process clinical features
            selected_clinical_df = geo_select_clinical_features(
                clinical_df=clinical_data,
                trait=trait,
                trait_row=trait_row,
                convert_trait=convert_trait,
                age_row=age_row,
                convert_age=convert_age,
                gender_row=gender_row,
                convert_gender=convert_gender
            )
            
            # Preview the processed clinical data
            preview = preview_df(selected_clinical_df)
            print("Preview of selected clinical features:")
            print(preview)
            
            # Save the processed clinical data
            os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
            selected_clinical_df.to_csv(out_clinical_data_file)
            print(f"Clinical data saved to {out_clinical_data_file}")
        else:
            print("No series matrix file found. Cannot extract clinical features.")
    except Exception as e:
        print(f"Error processing clinical data: {e}")
        print(f"Error traceback: {traceback.format_exc()}")
        print("Unable to extract clinical features.")


Available files in ../../input/GEO/Asthma/GSE205151: ['GSE205151_family.soft.gz', 'GSE205151_series_matrix.txt.gz']
Found series matrix file: ../../input/GEO/Asthma/GSE205151/GSE205151_series_matrix.txt.gz
Extracted sample characteristics:
Row 0: ['"polyic_stimulation: Stimulated"', '"polyic_stimulation: Unstimulated"', '"polyic_stimulation: No"']
Row 1: ['""', '"cluster: 1"', '"cluster: 2"']
Preview of selected clinical features:
{'"GSM6205808"': [nan], '"GSM6205809"': [nan], '"GSM6205810"': [nan], '"GSM6205811"': [nan], '"GSM6205812"': [nan], '"GSM6205813"': [nan], '"GSM6205814"': [nan], '"GSM6205815"': [nan], '"GSM6205816"': [nan], '"GSM6205817"': [nan], '"GSM6205818"': [nan], '"GSM6205819"': [nan], '"GSM6205820"': [nan], '"GSM6205821"': [nan], '"GSM6205822"': [nan], '"GSM6205823"': [nan], '"GSM6205824"': [nan], '"GSM6205825"': [nan], '"GSM6205826"': [nan], '"GSM6205827"': [nan], '"GSM6205828"': [nan], '"GSM6205829"': [nan], '"GSM6205830"': [nan], '"GSM6205831"': [nan], '"GSM6205832

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Asthma/GSE205151/GSE205151_series_matrix.txt.gz
Gene data shape: (608, 144)
First 20 gene/probe identifiers:
Index(['ABCB1', 'ABCF1', 'ABL1', 'ADA', 'AHR', 'AICDA', 'AIRE', 'ALAS1', 'APP',
       'ARG1', 'ARG2', 'ARHGDIB', 'ATG10', 'ATG12', 'ATG16L1', 'ATG5', 'ATG7',
       'ATM', 'B2M', 'B3GAT1'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers from the previous step
# The identifiers appear to be official human gene symbols (e.g., ABCB1, ABCF1, ABL1)
# These are proper human gene symbols that don't require further mapping

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# First, re-extract the necessary files from the cohort directory
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Get the gene data again
gene_data = get_genetic_data(matrix_file)

# Read background information and clinical data again to ensure we have the correct data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Save the gene data (no normalization needed as the gene symbols are already standard)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")

# Using the correct trait_row identified in step 2
# Using the correct convert_trait function from step 2
def convert_trait(value):
    if pd.isna(value):
        return None
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    # Convert to binary (0 for cluster 1, 1 for cluster 2)
    if value == '1':
        return 0
    elif value == '2':
        return 1
    else:
        return None

# Extract clinical features using the appropriate conversion functions
selected_clinical_data = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=1,  # Using trait_row = 1 for cluster as identified in step 2
    convert_trait=convert_trait,
    age_row=None,  # No age data available
    convert_age=None,
    gender_row=None,  # No gender data available
    convert_gender=None
)

# Save the processed clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_data.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_data, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5] if not linked_data.empty else "Linked data is empty")

# Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# Check for bias in features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from neutrophils with cluster information indicating response patterns to viral stimuli in children with critical asthma."
)

# Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for analysis. No linked data file saved.")

Gene data saved to ../../output/preprocess/Asthma/gene_data/GSE205151.csv
Clinical data saved to ../../output/preprocess/Asthma/clinical_data/GSE205151.csv
Linked data shape: (144, 609)
Linked data preview (first 5 rows, 5 columns):
            Asthma  ABCB1  ABCF1  ABL1   ADA
GSM6205808     0.0    5.0   21.0  49.0  27.0
GSM6205809     0.0    3.0   16.0   5.0   3.0
GSM6205810     1.0    3.0   15.0   3.0  15.0
GSM6205811     1.0    5.0   15.0   7.0  19.0
GSM6205812     0.0    5.0   17.0   2.0   8.0
Data shape after handling missing values: (143, 609)
For the feature 'Asthma', the least common label is '0.0' with 69 occurrences. This represents 48.25% of the dataset.
The distribution of the feature 'Asthma' in this dataset is fine.



Linked data saved to ../../output/preprocess/Asthma/GSE205151.csv
